In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np

In [2]:
def make_env():
    # HalfCheetah-v5 è l’ambiente più recente in Gymnasium.
    # Se hai un’altra versione, per esempio con Gym classico, potresti dover usare "HalfCheetah-v2" o "HalfCheetah-v4".
    return gym.make("HalfCheetah-v5", reset_noise_scale=0.1, render_mode=None)

In [3]:
# 1. Creiamo un Vectorized Environment
env = DummyVecEnv([make_env])  
# 2. (Opzionale ma consigliato) Normalizziamo osservazioni e reward
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [4]:
# 3. Definiamo il modello RL (PPO)
model = PPO(
    policy="MlpPolicy", 
    env=env, 
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.0,
    verbose=0,
    # tensorboard_log="./ppo_halfcheetah_tensorboard/"
)

In [5]:
# 4. Alleniamo il modello
total_timesteps = 1_000_000  # puoi aumentare questo valore
model.learn(total_timesteps=total_timesteps)


In [6]:
#5. Salviamo il modello
model.save("ppo_halfcheetah_model")
env.save("vecnormalize_halfcheetah.pkl")  # salviamo anche i parametri di normalizzazione

In [7]:
# 6. Valutazione
#    Ora creiamo un nuovo ambiente per il rendering
eval_env = gym.make("HalfCheetah-v5", reset_noise_scale=0.1)
                    # , render_mode="human")

In [8]:
# Ricreiamo il wrapper di normalizzazione con i parametri salvati
eval_env = DummyVecEnv([lambda: eval_env])
eval_env = VecNormalize.load("vecnormalize_halfcheetah.pkl", eval_env)
eval_env.training = False
eval_env.norm_reward = False  # In valutazione di solito non normalizziamo la reward


In [9]:
obs = eval_env.reset()
episode_reward = 0.0
done = False

while not done:
    # azioni deterministiche in valutazione
    action, _states = model.predict(obs, deterministic=True)
    # obs, reward, terminated, truncated, info = eval_env.step(action)
    observation, reward, terminated, truncated = eval_env.step(action)
    episode_reward += reward[0]  # reward è un array nel vectorized env
    done = terminated or truncated

print("Ricompensa totale episodio:", episode_reward)
eval_env.close()

Ricompensa totale episodio: 0.20572304725646973
